# Homework #3: PCA/Hyperparameter/CV
Data source: http://archive.ics.uci.edu/ml/datasets/Polish+companies+bankruptcy+data

In [80]:
from scipy.io import arff
import pandas as pd
import numpy as np

In [81]:
data = arff.loadarff('4year.arff')
df = pd.DataFrame(data[0])

In [82]:
df['bankruptcy'] = (df['class']==b'1')
df.drop(columns=['class'], inplace=True)
df.columns = ['X{0:02d}'.format(k) for k in range(1,65)] + ['bankruptcy']

In [83]:
df.describe()

,X01,X02,X03,X04,X05,X06,X07,X08,X09,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41,X42,X43,X44,X45,X46,X47,X48,X49,X50,X51,X52,X53,X54,X55,X56,X57,X58,X59,X60,X61,X62,X63,X64
count,9791.000000,9791.000000,9791.000000,9749.000000,9.771000e+03,9791.000000,9791.000000,9773.000000,9792.000000,9791.000000,9791.000000,9749.000000,9771.000000,9791.000000,9.784000e+03,9773.000000,9773.000000,9791.000000,9771.000000,9771.000000,9634.000000,9791.000000,9771.000000,9581.000000,9791.000000,9773.000000,9.151000e+03,9561.000000,9791.000000,9771.000000,9771.000000,9696.000000,9749.000000,9773.000000,9791.000000,9791.000000,5350.000000,9791.000000,9771.000000,9749.000000,9605.000000,9771.000000,9.771000e+03,9.771000e+03,9179.000000,9749.000000,9719.000000,9791.000000,9771.000000,9773.000000,9791.000000,9716.000000,9561.000000,9561.000000,9.792000e+03,9771.000000,9791.000000,9776.000000,9791.000000,9178.000000,9760.000000,9.771000e+03,9749.000000,9561.000000
mean,0.043019,0.596404,0.130959,8.136600,6.465164e+01,-0.059273,0.059446,19.884016,1.882296,0.389040,0.075417,0.210989,0.398902,0.059460,3.017681e+03,0.617918,20.976033,0.064580,-0.019081,62.704589,1.218724,0.066203,-0.070364,0.247742,0.222839,0.451115,1.115883e+03,6.725180,3.946479,5.353531,0.041258,341.625124,8.445313,4.979157,0.058091,2.077261,70.659877,0.487190,-1.072578,3.064235,0.968902,-0.371479,7.356944e+02,6.729892e+02,5.458024,7.274189,112.989701,-0.002370,-0.517222,7.085001,0.469319,10.031638,6.114681,7.402928,7.686330e+03,-0.992263,0.035022,1.133287,0.856053,118.156064,25.194430,2.015157e+03,8.660813,35.949619
std,0.359321,4.587122,4.559074,290.647281,1.475939e+04,6.812754,0.533344,698.697015,17.674650,4.590299,0.528232,74.237274,37.974787,0.533344,1.022731e+05,78.494223,698.757245,0.736143,25.583613,377.204157,5.930840,0.504481,23.889882,8.268015,4.852418,74.037751,3.143938e+04,147.963574,0.865714,340.974268,25.585724,6145.604519,69.690183,58.480776,0.483463,17.341615,621.311292,4.578432,77.056762,87.916989,41.191681,14.174896,3.283705e+04,3.281128e+04,186.414617,290.619843,1993.125597,0.525467,15.737098,287.770829,4.554869,897.307846,90.190534,146.013868,7.605261e+04,77.007971,8.945365,8.038201,26.393305,3230.316692,1099.260821,1.171461e+05,60.838202,483.318623
min,-12.458000,0.000000,-445.910000,-0.045319,-3.794600e+05,-486.820000,-12.458000,-1.848200,-0.032371,-445.910000,-12.244000,-6331.800000,-1460.600000,-12.458000,-1.567500e+06,-6331.800000,0.000857,-12.458000,-1578.700000,0.000000,-1.146300,-12.244000,-1578.700000,-314.370000,-466.340000,-6331.800000,-2.590100e+05,-990.020000,-0.440090,-4940.000000,-1495.600000,0.000000,0.000000,-756.500000,-9.043100,-0.000014,-3.715000,-445.910000,-7522.000000,-8.833300,-1086.800000,-719.800000,-1.158700e+05,-1.158700e+05,-2834.900000,-6.639200,-3.630700,-13.815000,-837.860000,-0.045239,0.000000,0.000000,-1033.700000,-1033.700000,-7.132200e+05,-7522.100000,-597.420000,-30.892000,-284.380000,0.000000,-12.656000,-1.496500e+04,-0.024390,-0.000015
25%,0.001321,0.263145,0.020377,1.047000,-5.121700e+01,-0.000578,0.003004,0.428300,1.006675,0.294440,0.009457,0.007608,0.021204,0.003008,2.173550e+02,0.061874,1.448900,0.003008,0.001937,15.244500,0.920125,0.000000,0.000963,0.004049,0.135600,0.057772,0.000000e+00,0.035097,3.398450,0.082730,0.004434,47.222750,2.702300,0.297400,0.000872,1.030900,1.097050,0.419750,0.000687,0.051234,0.025029,0.000000,6.890100e+01,3.651950e+01,0.010131,0.613890,15.836000,-0.047398,-0.035386,0.768760,0.185285,0.128978,0.684900,0.946990,2.184000e+01,0.003121,0.008768,0.885722,0.000000,5.356325,4.267700,4.323400e+01,2.938800,2.012900
50%,0.041364,0.467740,0.199290,1.591800,-5.557600e-02,0.000000,0.048820,1.088700,1.161300,0.510450,0.062544,0.143690,0.063829,0.048859,9.065550e+02,0.219130,2.134600,0.048859,0.030967,35.657000,1.045700,0.050118,0.025931,0.149150,0.386750,0.199240,1.005500e+00,0.470770,3.976400,0.226950,0.037939,80.884500,4.467

In [84]:
sum(df.bankruptcy == True)

515

In [85]:
df.fillna(df.mean(), inplace=True)
df.isna().sum()
X_imp = df.values

In [86]:
from sklearn.model_selection import train_test_split

X, y = X_imp[:, :-1], X_imp[:, -1]
y = y.astype(bool)
X_train, X_test, y_train, y_test =\
    train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)

In [87]:
import sklearn.preprocessing as skpre

stdsc = skpre.StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
print(X_train_std.shape)
X_test_std = stdsc.transform(X_test)
print(X_test_std.shape)

(6854, 64)
(2938, 64)


# Dependencies

In [88]:
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix

# Pipeline of:
1. PCA
1. model built by grid search

## Tricks:
Since the dataset is very imbalanced:
1. using "class_weight='balanced'" to adjust weights of observations, and
2. using F1 score metric.

## (1) LR pipeline
hyper-parameter:
1. C: np.logspace(-6,6,100)
2. penalty: "none", "l1", "l2"

### (a) LR with penalties

In [89]:
pipe_lr = make_pipeline(
    PCA(n_components=3),
    GridSearchCV( # grid search for hyper-parameters
        LogisticRegression(
            solver='liblinear',
            class_weight='balanced',
            random_state=1,
        ), 
        param_grid={
            'C': np.logspace(-6,6,100),
            "penalty": ["l1", "l2"]
        }, 
        cv=5, # 5 fold Cross-Validation
        scoring='f1',
        verbose=1
    )
)

pipe_lr.fit(X_train_std, y_train)
param_best_lr = pipe_lr.steps[1][1].best_params_
y_train_pred = pipe_lr.predict(X_train_std)
y_test_pred = pipe_lr.predict(X_test_std)
f1_train_lr = f1_score(y_train, y_train_pred)
f1_test_lr = f1_score(y_test, y_test_pred)
acc_train_lr = accuracy_score(y_train, y_train_pred)
acc_test_lr = accuracy_score(y_test, y_test_pred)
print("Best hyper-parameters: {}\nTraining F1: {:.4f}, accuracy: {:.4f}\nTesting F1: {:.4f}, accuracy: {:.4f}".format(
    param_best_lr,
    f1_train_lr,
    acc_train_lr,
    f1_test_lr,
    acc_test_lr
))

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Best hyper-parameters: {'C': 0.07054802310718646, 'penalty': 'l2'}
Training F1: 0.1792, accuracy: 0.7286
Testing F1: 0.1800, accuracy: 0.7270


[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    5.6s finished


### (b) LR with no penalty

In [90]:
pipe_lr2 = make_pipeline(
    PCA(n_components=3),
    LogisticRegression(
        class_weight='balanced',
        random_state=1,
    ), 
)

pipe_lr2.fit(X_train_std, y_train)
y_train_pred = pipe_lr2.predict(X_train_std)
y_test_pred = pipe_lr2.predict(X_test_std)
f1_train_lr2 = f1_score(y_train, y_train_pred)
f1_test_lr2 = f1_score(y_test, y_test_pred)
acc_train_lr2 = accuracy_score(y_train, y_train_pred)
acc_test_lr2 = accuracy_score(y_test, y_test_pred)
print("Training F1: {:.4f}, accuracy: {:.4f}\nTesting F1: {:.4f}, accuracy: {:.4f}".format(
    f1_train_lr2,
    acc_train_lr2,
    f1_test_lr2,
    acc_test_lr2
))

Training F1: 0.1798, accuracy: 0.7298
Testing F1: 0.1800, accuracy: 0.7270


**Conclusion: LR with no penalty is OK.**

## (2) SVM pipeline
hyper-parameter:
1. C: np.logspace(-2,2,4)
2. gamma: np.logspace(-2,2,4) * 0.5

In [91]:
pipe_svm = make_pipeline(
    PCA(n_components=3),
    GridSearchCV(
        SVC(
            kernel='rbf', 
            tol=1e-2,
            class_weight='balanced',
            random_state=1,
        ), 
        param_grid={
            'C': np.logspace(-2,2,4),
            'gamma': np.logspace(-2,2,4) * 0.5,
        }, 
        cv=5, 
        scoring='f1',
        verbose=1
    )
)

pipe_svm.fit(X_train_std, y_train)
param_best_svm = pipe_svm.steps[1][1].best_params_
y_train_pred = pipe_svm.predict(X_train_std)
y_test_pred = pipe_svm.predict(X_test_std)
f1_train_svm = f1_score(y_train, y_train_pred)
f1_test_svm = f1_score(y_test, y_test_pred)
acc_train_svm = accuracy_score(y_train, y_train_pred)
acc_test_svm = accuracy_score(y_test, y_test_pred)
print("Best hyper-parameters: {}\nTraining F1: {:.4f}, accuracy: {:.4f}\nTesting F1: {:.4f}, accuracy: {:.4f}".format(
    param_best_svm,
    f1_train_svm,
    acc_train_svm,
    f1_test_svm,
    acc_test_svm
))


Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:  1.3min finished


Best hyper-parameters: {'C': 0.21544346900318834, 'gamma': 0.005}
Training F1: 0.1860, accuracy: 0.7345
Testing F1: 0.1866, accuracy: 0.7359


## (3) Decision Tree pipeline
hyper-parameter:
1. criterion: ["gini", "entropy"]
2. max_depth: range(1, 10)

In [92]:
pipe_tree = make_pipeline(
    PCA(n_components=3),
    GridSearchCV(
        DecisionTreeClassifier(
            class_weight='balanced',
            random_state=1,
        ), 
        param_grid={
            "criterion": ["gini", "entropy"],
            "max_depth": range(1, 10)
        }, 
        cv=5, 
        scoring='f1',
        verbose=1
    )
)

pipe_tree.fit(X_train_std, y_train)
param_best_tree = pipe_tree.steps[1][1].best_params_
y_train_pred = pipe_tree.predict(X_train_std)
y_test_pred = pipe_tree.predict(X_test_std)
f1_train_tree = f1_score(y_train, y_train_pred)
f1_test_tree = f1_score(y_test, y_test_pred)
acc_train_tree = accuracy_score(y_train, y_train_pred)
acc_test_tree = accuracy_score(y_test, y_test_pred)
print("Best hyper-parameters: {}\nTraining F1: {:.4f}, accuracy: {:.4f}\nTesting F1: {:.4f}, accuracy: {:.4f}".format(
    param_best_tree,
    f1_train_tree,
    acc_train_tree,
    f1_test_tree,
    acc_test_tree
))

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Best hyper-parameters: {'criterion': 'gini', 'max_depth': 3}
Training F1: 0.2015, accuracy: 0.7213
Testing F1: 0.1873, accuracy: 0.7223


[Parallel(n_jobs=1)]: Done  90 out of  90 | elapsed:    1.2s finished


# Additional
prove F1 is better metrics in this imbalanced situation comparing with Accuracy.

In [97]:
pipe_tree2 = make_pipeline(
    PCA(n_components=3),
    DecisionTreeClassifier(
        class_weight='balanced',
        random_state=1,
    ), 
)

pipe_tree2.fit(X_train_std, y_train)

y_train_pred = pipe_tree2.predict(X_train_std)
y_test_pred = pipe_tree2.predict(X_test_std)
f1_train_tree2 = f1_score(y_train, y_train_pred)
f1_test_tree2 = f1_score(y_test, y_test_pred)
acc_train_tree2 = accuracy_score(y_train, y_train_pred)
acc_test_tree2 = accuracy_score(y_test, y_test_pred)

print("Training F1: {:.4f}, accuracy: {:.4f}\nTesting F1: {:.4f}, accuracy: {:.4f}".format(
    f1_train_tree2,
    acc_train_tree2,
    f1_test_tree2,
    acc_test_tree2
))

Training F1: 1.0000, accuracy: 1.0000
Testing F1: 0.0929, accuracy: 0.9003


Although the accuracy scores are very high, the F1 score of testing set is extremely low.
Let's see the confusion metrix.

In [105]:
y_test_pred_tree = pipe_tree.predict(X_test_std)
y_test_pred_tree2 = pipe_tree2.predict(X_test_std)
confusion_mtx = confusion_matrix(y_test, y_test_pred_tree)
confusion_mtx2 = confusion_matrix(y_test, y_test_pred_tree2)
prec_ratio = confusion_mtx[-1, -1] / np.sum(confusion_mtx[-1, :])
prec_ratio2 = confusion_mtx2[-1, -1] / np.sum(confusion_mtx2[-1, :])
recall_ratio = confusion_mtx[-1, -1] / np.sum(confusion_mtx[:, -1])
recall_ratio2 = confusion_mtx2[-1, -1] / np.sum(confusion_mtx2[:, -1])

print("""
# tree trained under F1 metric:
Testing F1: {:.4f}, accuracy: {:.4f}
Confusion_matrix: 
{}
Precision: {:.4f}
Recall: {:.4f}

# tree trained under accuracy metric:
Testing F1: {:.4f}, accuracy: {:.4f}
Confusion_matrix: 
{}
Precision: {:.4f}
Recall: {:.4f}
""".format(
    f1_test_tree,
    acc_test_tree,
    confusion_mtx,
    prec_ratio,
    recall_ratio,
    f1_test_tree2,
    acc_test_tree2,
    confusion_mtx2,
    prec_ratio2,
    recall_ratio2,
))


# tree trained under F1 metric:
Testing F1: 0.1873, accuracy: 0.7223
Confusion_matrix: 
[[2028  755]
 [  61   94]]
Precision: 0.6065
Recall: 0.1107

# tree trained under accuracy metric:
Testing F1: 0.0929, accuracy: 0.9003
Confusion_matrix: 
[[2630  153]
 [ 140   15]]
Precision: 0.0968
Recall: 0.0893



We can see that the precision ratio of tree trained under accuracy metric is significantly lower.